# Import Libraries

In [1]:
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# import necessary packages
import pandas as pd
from copy import deepcopy as copy
import numpy as np
from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn
from sklearn import datasets
from sklearn import preprocessing

# BPNN preparation

In [27]:
sablayan_clustered_df = pd.read_csv('Sablayan_Data_Clustered.csv')
sablayan_clustered_df

,Unnamed: 0,id,BARANGAY,PSGC,LAND COVERAGE,RHU,POPULATION DENSITY PER HEXAGON,POIs PER HEXAGON,ROADS,STORM SURGE HAZARD LEVEL,FLOOD HAZARD LEVEL,LANDSLIDE HAZARD LEVEL,LANDCOVER NUMERICAL,HCFAI,Cluster,Unnamed: 15
0,0.0,22.0,Ligaya,PH175109009,Brush/Shrubs,0.0,0.000000,1.0,1.0,0.0,0.0,2.0,1.0,0.000000,LOW,0
1,10.0,23.0,Burgos,PH175109009,Brush/Shrubs,0.0,0.000000,0.0,0.0,0.0,0.0,2.0,1.0,0.000000,LOW,0
2,20.0,49.0,Pag-Asa,PH175109004,NaN,0.0,154.538807,0.0,0.0,3.0,2.0,0.0,0.0,1151.197170,MID,1
3,21.0,50.0,San Agustin,PH175109004,Built Up,0.0,214.812672,0.0,0.0,3.0,2.0,0.0,0.0,1600.191852,MID,1
4,22.0,51.0,San Agustin,PH175109004,Grassland,0.0,302.902705,0.0,0.0,3.0,1.0,0.0,1.0,2469.775420,HIGH,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
968,1060.0,1363.0,Pag-Asa,PH175109003,Brush/Shrubs,0.0,16.884621,0.0,0.0,0.0,0.0,3.0,1.0,235.553869,LOW,0
969,1061.0,1364.0,Pag-Asa,PH175109003,NaN,0.0,21.536779,0.0,0.0,0.0,0.0,3.0,0.0,293.994123,LOW,0
970,1062.0,1397.0,Pag-Asa,PH175109003,NaN,0.0,0.000000,0.0,0.0,0.0,0.0,3.0,0.0,0.000000,LOW,0
971,1072.0,1398.0,Pag-Asa,PH175109003,NaN,0.0,0.000000,0.0,0.0,0.0,0.0,3.0,0.0,0.000000,LOW,0


In [28]:
edited_df = sablayan_clustered_df.drop(['id', 'BARANGAY', 'PSGC', 'LAND COVERAGE', 'Cluster', 'Unnamed: 0'], axis=1)

In [29]:
edited_df

,RHU,POPULATION DENSITY PER HEXAGON,POIs PER HEXAGON,ROADS,STORM SURGE HAZARD LEVEL,FLOOD HAZARD LEVEL,LANDSLIDE HAZARD LEVEL,LANDCOVER NUMERICAL,HCFAI,Unnamed: 15
0,0.0,0.000000,1.0,1.0,0.0,0.0,2.0,1.0,0.000000,0
1,0.0,0.000000,0.0,0.0,0.0,0.0,2.0,1.0,0.000000,0
2,0.0,154.538807,0.0,0.0,3.0,2.0,0.0,0.0,1151.197170,1
3,0.0,214.812672,0.0,0.0,3.0,2.0,0.0,0.0,1600.191852,1
4,0.0,302.902705,0.0,0.0,3.0,1.0,0.0,1.0,2469.775420,2
...,...,...,...,...,...,...,...,...,...,...
968,0.0,16.884621,0.0,0.0,0.0,0.0,3.0,1.0,235.553869,0
969,0.0,21.536779,0.0,0.0,0.0,0.0,3.0,0.0,293.994123,0
970,0.0,0.000000,0.0,0.0,0.0,0.0,3.0,0.0,0.000000,0
971,0.0,0.000000,0.0,0.0,0.0,0.0,3.0,0.0,0.000000,0


In [30]:
edited_df.rename(columns = {'Unnamed: 15':'Cluster'}, inplace = True)

In [31]:
# Removing mid values in Cluster column
# 1. Remove rows with Cluter values of 1
# 2. Rename clusters 2 to 1
edited_df.drop(edited_df[edited_df['Cluster'] == 1].index, inplace = True)
edited_df['Cluster'] = edited_df['Cluster'].replace(2, 1)
edited_df


,RHU,POPULATION DENSITY PER HEXAGON,POIs PER HEXAGON,ROADS,STORM SURGE HAZARD LEVEL,FLOOD HAZARD LEVEL,LANDSLIDE HAZARD LEVEL,LANDCOVER NUMERICAL,HCFAI,Cluster
0,0.0,0.000000,1.0,1.0,0.0,0.0,2.0,1.0,0.000000,0
1,0.0,0.000000,0.0,0.0,0.0,0.0,2.0,1.0,0.000000,0
4,0.0,302.902705,0.0,0.0,3.0,1.0,0.0,1.0,2469.775420,1
5,0.0,1243.892810,13.0,1.0,3.0,2.0,2.0,0.0,5967.815673,1
6,0.0,0.000000,0.0,0.0,3.0,2.0,2.0,0.0,0.000000,0
...,...,...,...,...,...,...,...,...,...,...
968,0.0,16.884621,0.0,0.0,0.0,0.0,3.0,1.0,235.553869,0
969,0.0,21.536779,0.0,0.0,0.0,0.0,3.0,0.0,293.994123,0
970,0.0,0.000000,0.0,0.0,0.0,0.0,3.0,0.0,0.000000,0
971,0.0,0.000000,0.0,0.0,0.0,0.0,3.0,0.0,0.000000,0


In [32]:
x = edited_df
x_normalized = (x - x.min()) / (x.max() - x.min())
x_normalized

,RHU,POPULATION DENSITY PER HEXAGON,POIs PER HEXAGON,ROADS,STORM SURGE HAZARD LEVEL,FLOOD HAZARD LEVEL,LANDSLIDE HAZARD LEVEL,LANDCOVER NUMERICAL,HCFAI,Cluster
0,0.0,0.000000,0.021277,1.0,0.0,0.000000,0.666667,1.0,0.000000,0.0
1,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.666667,1.0,0.000000,0.0
4,0.0,0.171734,0.000000,0.0,1.0,0.333333,0.000000,1.0,0.164751,1.0
5,0.0,0.705237,0.276596,1.0,1.0,0.666667,0.666667,0.0,0.398096,1.0
6,0.0,0.000000,0.000000,0.0,1.0,0.666667,0.666667,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...
968,0.0,0.009573,0.000000,0.0,0.0,0.000000,1.000000,1.0,0.015713,0.0
969,0.0,0.012210,0.000000,0.0,0.0,0.000000,1.000000,0.0,0.019611,0.0
970,0.0,0.000000,0.000000,0.0,0.0,0.000000,1.000000,0.0,0.000000,0.0
971,0.0,0.000000,0.000000,0.0,0.0,0.000000,1.000000,0.0,0.000000,0.0


In [33]:
# Remove column name 'A'
x_normalized.drop(['Cluster'], axis=1)
x_normalized

,RHU,POPULATION DENSITY PER HEXAGON,POIs PER HEXAGON,ROADS,STORM SURGE HAZARD LEVEL,FLOOD HAZARD LEVEL,LANDSLIDE HAZARD LEVEL,LANDCOVER NUMERICAL,HCFAI,Cluster
0,0.0,0.000000,0.021277,1.0,0.0,0.000000,0.666667,1.0,0.000000,0.0
1,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.666667,1.0,0.000000,0.0
4,0.0,0.171734,0.000000,0.0,1.0,0.333333,0.000000,1.0,0.164751,1.0
5,0.0,0.705237,0.276596,1.0,1.0,0.666667,0.666667,0.0,0.398096,1.0
6,0.0,0.000000,0.000000,0.0,1.0,0.666667,0.666667,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...
968,0.0,0.009573,0.000000,0.0,0.0,0.000000,1.000000,1.0,0.015713,0.0
969,0.0,0.012210,0.000000,0.0,0.0,0.000000,1.000000,0.0,0.019611,0.0
970,0.0,0.000000,0.000000,0.0,0.0,0.000000,1.000000,0.0,0.000000,0.0
971,0.0,0.000000,0.000000,0.0,0.0,0.000000,1.000000,0.0,0.000000,0.0


In [64]:
y = edited_df['Cluster'].values
y

array([0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [35]:
num_features = len(x.columns)
print("Number of Features: {}".format(num_features))

Number of Features: 10


In [36]:
columns = []

for i in range(num_features):
    columns.append("x{}".format(i))

x_normalized.columns = columns # replacing the name of the columns
x_normalized

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9
0,0.0,0.000000,0.021277,1.0,0.0,0.000000,0.666667,1.0,0.000000,0.0
1,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.666667,1.0,0.000000,0.0
4,0.0,0.171734,0.000000,0.0,1.0,0.333333,0.000000,1.0,0.164751,1.0
5,0.0,0.705237,0.276596,1.0,1.0,0.666667,0.666667,0.0,0.398096,1.0
6,0.0,0.000000,0.000000,0.0,1.0,0.666667,0.666667,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...
968,0.0,0.009573,0.000000,0.0,0.0,0.000000,1.000000,1.0,0.015713,0.0
969,0.0,0.012210,0.000000,0.0,0.0,0.000000,1.000000,0.0,0.019611,0.0
970,0.0,0.000000,0.000000,0.0,0.0,0.000000,1.000000,0.0,0.000000,0.0
971,0.0,0.000000,0.000000,0.0,0.0,0.000000,1.000000,0.0,0.000000,0.0


In [65]:
# make y its own dataframe
# Getting the hot encoding for y (making it its own df)
# y = pd.get_dummies(y, prefix='y')
y

array([0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [66]:
y = pd.DataFrame(y, columns = ['Y_Cluster'])

print(y)
print(type(y))

     Y_Cluster
0            0
1            0
2            1
3            1
4            0
..         ...
852          0
853          0
854          0
855          0
856          0

[857 rows x 1 columns]
<class 'pandas.core.frame.DataFrame'>


# BPNN From Scratch

In [ ]:
# import necessary packages
import pandas as pd
from copy import deepcopy as copy
import numpy as np
from sklearn.metrics import accuracy_score

In [ ]:
# set the precision of numpy output to 3
np.set_printoptions(precision=3)


class OurNeuralNet(object):
    def __init__(self, X, y, layers, m, learning_rate=0.01, factor=0.01):
        """
        Our init values - X (the input), y (target values), m (number of examples),
        learning_rate, factor (determines how small the parameters are)
        """
        self.X = X
        self.layers = layers
        self.y = y 
        self.learning_rate=learning_rate
        self.factor = factor
        self.m = m
    @staticmethod
    def sigmoid(x):
        """
        Argument: value(s) x
        Returns: 1 / (1 + e^(-x)) - the sigmoid activation
        """ 
        return 1 / (1 + np.exp(-x))

    def parameters_initialization(self):
        """
        Argument:
        - picked from the __init__() magic function
        Returns:
        parameters dictionary -- contains initial parameter values:
        w1 - weight matrix of shape (n1, n0)
        b1 - bias vector of shape (n1, 1)
        w2 - weight matrix of shape (n2, n1)
        b2 - bias vector of shape (n2, 1), where
        ni - number of neurons at the ith layer.
        """    
        
        # Number of neurons in each layer. We just have 3 layers
        n0, n1, n2 = self.layers
        
        # defind the randomization seed - this ensures that the results are 
        # reproducible
        np.random.seed(3)
        
        # Generating parameter values for layer 1. 
        # factor determines how small or big the parameters are
        w1 = np.random.randn(n1,n0) * self.factor
        b1 = np.zeros((n1,1))
        
        # Generating initial parameters values for layer 2
        w2 = np.random.randn(n2,n1) * self.factor
        b2 = np.zeros((n2,1))

        parameters = {"w1": w1,
                        "b1": b1,
                            "w2": w2,
                                "b2": b2}
            

        return parameters
    
    def forward_propagation(self, parameters):
        """
        Argument: parameters - weights and biases for all the layers
        
        Returns:
        yhat - model output on one forward pass for all the training examples
        layer_ouputs - a dictionary containing model outputs at each layer.
        """
        
        # Pick the parameters w1, b1, w2 and b2 from parameters dictionary.
        w1 = parameters["w1"]
        b1 = parameters["b1"]
        w2 = parameters["w2"]
        b2 = parameters["b2"]

        # Perform computations for the hidden layer.
        # Ouput vector is named f1
        z1 = np.dot(w1, self.X) + b1
        f1 = self.sigmoid(z1)

        # Computations on the second/output layer.
        # Output is yhat
        z2 = np.dot(w2, f1) + b2
        yhat = self.sigmoid(z2)
    
        # Just to make sure that the output is of the dimension
        # we expect
        # It should be a vector of the predictions for the for all examples
        # self.X.shape[1] - number of training examples
        assert(yhat.shape == (1, self.X.shape[1]))
        # z-values are weighted input plus bias
        # f1 and yhat are the outputs of hidden and output layers after applying
        # Sigmoid activation
        layer_outputs = {"z1": z1,
                                "f1": f1,
                                    "z2": z2,
                                        "yhat": yhat}
    
        return yhat, layer_outputs
   
    def compute_cost(self, yhat, y):
        """
        Computes the cross-entropy cost of approximation
        Arguments:
        y_hat - Output of forward propagation. Its order is (1, # trainingexamples)
        y - vector of true values. Dimension is (1, # training examples)
        Returns:
        the cost
        """
        # crosss entropy loss function
        # np.log computes ln (natural log) - log to base 2
        
        cost = -np.sum(np.multiply(y, np.log(yhat)) + np.multiply(1-y, np.log(1-yhat)))/self.m
        
        # np.squeeze makes sure cost is the dimension we expect. 
        # E.g., turns [[17]] into 17 
        cost = float(np.squeeze(cost)) 
        return cost

    def backward_propagation(self, parameters, layers_output):
        """
        Arguments:
        parameters -- python dictionary containing our parameters 
        layers_output -- a dictionary containing "Z1", "f1", "Z2" and "yhat".
    
        Returns:
        gradients -- python dictionary containing our gradients with respect to different parameters
        """
    
        # First, retrieve w1 and w1 from the dictionary "parameters".
        w1 = parameters["w1"]
        w2 = parameters["w2"]
        
        # Retrieve also f1 and yhat from dictionary "layers_output".
        # f1 - output of neurons on hidden layer
        # yhat - output from the output neuron.
        f1 = layers_output["f1"]
        yhat = layers_output["yhat"]

        # Backward propagation gradients
        dz2 = yhat-self.y
        dw2 = np.dot(dz2, f1.T)/self.m
        # sum along columns (axis=1)
        # keepdims= Truethe axes which are reduced are left in 
        # the result as dimensions with size one. This allows for correct
        # array broadcasting
        db2 = 1/self.m * np.sum(dz2, axis=1, keepdims=True)
        dz1 = np.dot(w2.T, dz2) * (1-f1)*f1
        dw1 = np.dot(dz1, X.T)/self.m
        db1 = 1/self.m * np.sum(dz1, axis=1, keepdims=True)
        dw2 = np.dot(dz2, f1.T)/self.m

        gradients = {"dw1": dw1,
                            "db1": db1,
                                "dw2": dw2,
                                    "db2": db2}

        return gradients
    
    # GRADED FUNCTION: update_parameters

    def update_parameters(self, parameters, gradients):
        """
        Updates parameters using the gradient descent update rule discussed above
        Arguments:
        parameters -- python dictionary containing your parameters 
        gradients -- python dictionary containing your gradients 
        Returns:
        parameters -- python dictionary containing your updated parameters 
        """
        # Retrieve a copy of each parameter from the dictionary "parameters".
        w1 = copy(parameters["w1"])
        b1 = copy(parameters["b1"])
        w2 = copy(parameters["w2"])
        b2 = copy(parameters["b2"])

        # Retrieve each gradient from the dictionary "grads"
        dw1 = copy(gradients["dw1"])
        db1 = copy(gradients["db1"])
        dw2 = copy(gradients["dw2"])
        db2 = copy(gradients["db2"])

        ##  Update parameters with the update equations.
        # the hidden-input parameters
        w1 = w1 - self.learning_rate*dw1
        b1 = b1 - self.learning_rate*db1
        # the output-hidden parameters
        w2 = w2 - self.learning_rate*dw2
        b2 = b2 - self.learning_rate*db2

        parameters = {"w1": w1,
                          "b1": b1,
                              "w2": w2,
                                  "b2": b2}

        return parameters

In [ ]:
def train_model(n_iterations, X, y, layers, n_examples, lr, factor):
    """
    Arguments:
        n_iterations - number of training iterations,
        X - the input data (feature data),
        y - target/true values,
        layers - a tuple with the # neurons for each layer
        lr - the learning rate,
        factor -  user-defined value that determines how big initialized parameters are
    """
    #Initialize the class for OurNeuralNetwork
    s = OurNeuralNet(X=X, y=y, layers=layers,m=n_examples, learning_rate=lr, factor=factor)

    #Initialize parameters
    parameters = s.parameters_initialization()

    #number of iterations
    for iteration in range(n_iterations+1):
        # perform forward propagation
        # At the first iteration the parameters are the initialized ones.
        y_hat, layers_output = s.forward_propagation(parameters)
        # compute the loss using the output (yhat) and the true value (y) - 3 decimal places
        cost = round(s.compute_cost(y_hat, y), 3)
        # Compute all the required gradients through back-propagation.
        gradients = s.backward_propagation(parameters=parameters, layers_output=layers_output)
        # Update the parameters
        parameters=s.update_parameters(parameters,gradients)

        # we replace with values greater or equal to 0.5 with 1 and 0 otherwise
        predictions = np.select(
            [y_hat<0.5, y_hat>=0.5],      # list of conditions
            [0, 1])# list of corresponding values or computations)
        
        #compute accuracy to 3 decimal places
        accuracy = round(accuracy_score(np.squeeze(y), np.squeeze(predictions)), 3)
        if iteration%1000==0 or iteration==n_iterations:
            #print out cost and accuracy after every 1000 iterations and at the last iteration.
            print("iteration",iteration,"cost:-->", cost, "accuracy-->", accuracy)
        # After that we use the updated parameters to go through another iteration.
    # return the final parameters after model training
    return parameters

In [ ]:
# Load the data from online source
# df = pd.read_csv("https://kipronokoech.github.io/assets/datasets/marks.csv")

# use the first 300 examples for model training. We will use the remaining 95 for testing.
# df_train = df[:300]

X = x_normalized[:300]
y = y[:300]
# convert pandas dataframes and series to numpy array
# X = np.array(df_train.drop(["y"], axis=1)).T # feature matrix
# y = np.array(df_train["y"]).reshape(1, -1)

X= np.array(X).T # feature matrix
y = np.array(y).reshape(1, -1)


# Defining size of our layers
n0 = X.shape[0] #input size = number of features
n1 = 4 # 4 neurons on the hidden layer
n2 = y.shape[0] # one neuron for output layers

print(n0)
print(n2)

print(X)
print(y)

print(X.shape[0])
print(y.shape[0])

# number of training example is found at the 1st index of X.shape
n_examples = X.shape[1]

#Tuple of our layers.
layers = (n0, n1, n2)

#call the training model
parameters = train_model(n_iterations=4500, X=X, y=y, layers=layers, n_examples=n_examples, lr=0.2, factor=0.01)
print(parameters)

10
1
[[0.    0.    0.    ... 0.    0.    0.   ]
 [0.    0.    0.088 ... 0.005 0.006 0.005]
 [0.021 0.    0.    ... 0.    0.    0.   ]
 ...
 [1.    1.    0.    ... 1.    1.    0.   ]
 [0.    0.    0.077 ... 0.008 0.01  0.009]
 [0.    0.    0.5   ... 0.    0.    0.   ]]
[[0 0 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 0
  1 1 1 1 1 1 1 1 0 0 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 0 0 1 1 1 0 0 1 1 1
  0 1 1 1 1 1 1 0 1 1 0 1 1 0 0 1 1 0 0 1 0 1 1 1 1 1 1 0 0 1 1 0 1 1 0 1
  1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 0 1 1 0 0 1 0 0 1 1 0 0 0 0 1 1 0 0 1 1
  0 1 0 1 0 1 0 1 1 0 1 0 1 1 1 1 0 0 0 1 0 1 1 1 0 1 0 1 1 0 0 0 1 0 0 1
  0 1 1 0 1 1 1 1 0 1 1 0 0 1 1 1 1 1 1 1 1 1 1 0 1 0 0 1 1 0 0 1 1 0 1 1
  0 0 0 1 0 0 1 1 1 0 1 1 0 1 1 1 0 1 0 1 1 1 1 0 1 1 0 1 0 1 1 0 0 1 0 1
  0 0 1 0 1 1 1 0 1 0 1 0 0 1 1 1 1 0 0 1 1 1 1 1 0 0 0 1 0 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 0 0 1 1]]
10
1
iteration 0 cost:--> nan accuracy--> 0.33
iteration 1000 cost:--> nan accuracy--> 0.33
iteration 2000 cost

# BPNN Using PyTorch

In [14]:
import torch
import torch.nn as nn
from sklearn import datasets
from sklearn import preprocessing

# Set manual seed
torch.manual_seed(2)

In [15]:
# iris = datasets.load_iris()
# X = torch.tensor(preprocessing.normalize(iris.data[:, :2]), dtype=torch.float)
# y = torch.tensor(iris.target.reshape(-1, 1), dtype=torch.float)

X = torch.tensor(x_normalized.values, dtype=torch.float)
y = torch.tensor(y.values, dtype=torch.float)

print(X)
print(y)
print(X.size())
print(y.size())

tensor([[0.0000, 0.0000, 0.0213,  ..., 1.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 1.0000, 0.0000, 0.0000],
        [0.0000, 0.1717, 0.0000,  ..., 1.0000, 0.1648, 1.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [   nan,    nan,    nan,  ...,    nan,    nan, 0.0000]])
tensor([[0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],


In [16]:
# We only take 2 classes to make a binary classification problem
X = X[:y[y < 2].size()[0]]
y = y[:y[y < 2].size()[0]]

print(X.size())
print(y.size())

torch.Size([857, 10])
torch.Size([857, 1])


In [16]:
class FNN(nn.Module):
    def __init__(self, ):
        super().__init__()

        # Dimensions for input, hidden and output
        self.input_dim = 10 # 2
        self.hidden_dim = 120 #32
        self.output_dim = 21 # 1

        # Learning rate definition
        self.learning_rate = 0.001 # 0.001

        # Our parameters (weights)
        # w1: 2 x 32
        self.w1 = torch.randn(self.input_dim, self.hidden_dim)

        # w2: 32 x 1
        self.w2 = torch.randn(self.hidden_dim, self.output_dim)

    def sigmoid(self, s):
        return 1 / (1 + torch.exp(-s))

    def sigmoid_first_order_derivative(self, s):
        return s * (1 - s)

    # Forward propagation
    def forward(self, X):
        # First linear layer
        self.y1 = torch.matmul(X, self.w1) # 3 X 3 ".dot" does not broadcast in PyTorch

        # First non-linearity
        self.y2 = self.sigmoid(self.y1)

        # Second linear layer
        self.y3 = torch.matmul(self.y2, self.w2)

        # Second non-linearity
        y4 = self.sigmoid(self.y3)
        return y4

    # Backward propagation
    def backward(self, X, l, y4):
        # Derivative of binary cross entropy cost w.r.t. final output y4
        self.dC_dy4 = y4 - l

        '''
        Gradients for w2: partial derivative of cost w.r.t. w2
        dC/dw2
        '''
        self.dy4_dy3 = self.sigmoid_first_order_derivative(y4)
        self.dy3_dw2 = self.y2

        # Y4 delta: dC_dy4 dy4_dy3
        self.y4_delta = self.dC_dy4 * self.dy4_dy3

        # This is our gradients for w1: dC_dy4 dy4_dy3 dy3_dw2
        self.dC_dw2 = torch.matmul(torch.t(self.dy3_dw2), self.y4_delta)

        '''
        Gradients for w1: partial derivative of cost w.r.t w1
        dC/dw1
        '''
        self.dy3_dy2 = self.w2
        self.dy2_dy1 = self.sigmoid_first_order_derivative(self.y2)

        # Y2 delta: (dC_dy4 dy4_dy3) dy3_dy2 dy2_dy1
        self.y2_delta = torch.matmul(self.y4_delta, torch.t(self.dy3_dy2)) * self.dy2_dy1

        # Gradients for w1: (dC_dy4 dy4_dy3) dy3_dy2 dy2_dy1 dy1_dw1
        self.dC_dw1 = torch.matmul(torch.t(X), self.y2_delta)

        # Gradient descent on the weights from our 2 linear layers
        self.w1 -= self.learning_rate * self.dC_dw1
        self.w2 -= self.learning_rate * self.dC_dw2

    def train(self, X, l):
        # Forward propagation
        y4 = self.forward(X)

        # Backward propagation and gradient descent
        self.backward(X, l, y4)

In [21]:
# Instantiate our model class and assign it to our model object
model = FNN()

# Loss list for plotting of loss behaviour
loss_lst = []

# Number of times we want our FNN to look at all 100 samples we have, 100 implies looking through 100x
num_epochs = 101

# Let's train our model with 100 epochs
for epoch in range(num_epochs):
    # Get our predictions
    y_hat = model(X) # ⚠️ check this out. Look where nans are coming from ⚠️
    print("Y_hat: ", y_hat)

    # Cross entropy loss, remember this can never be negative by nature of the equation
    # But it does not mean the loss can't be negative for other loss functions
    cross_entropy_loss = -(y * torch.log(y_hat) + (1 - y) * torch.log(1 - y_hat))
    print("CEL:", cross_entropy_loss)

    # We have to take cross entropy loss over all our samples, 100 in this 2-class iris dataset
    mean_cross_entropy_loss = torch.mean(cross_entropy_loss).detach().item()

    # Print our mean cross entropy loss
    if epoch % 20 == 0:
        print('Epoch {} | Loss: {}'.format(epoch, mean_cross_entropy_loss))
    loss_lst.append(mean_cross_entropy_loss)

    # (1) Forward propagation: to get our predictions to pass to our cross entropy loss function
    # (2) Back propagation: get our partial derivatives w.r.t. parameters (gradients)
    # (3) Gradient Descent: update our weights with our gradients
    model.train(X, y)

Y_hat:  tensor([[2.5550e-01, 4.2182e-01, 1.0213e-02,  ..., 2.6194e-03, 8.1538e-01,
         9.9292e-01],
        [3.1758e-01, 4.4337e-02, 8.7526e-03,  ..., 2.3429e-03, 9.7781e-01,
         9.8971e-01],
        [9.5435e-01, 3.9157e-03, 2.0810e-02,  ..., 6.9055e-05, 9.8773e-01,
         9.0309e-01],
        ...,
        [3.3955e-01, 2.5881e-02, 1.3318e-02,  ..., 1.4753e-03, 8.5970e-01,
         6.9320e-01],
        [3.3954e-01, 2.5881e-02, 1.3318e-02,  ..., 1.4753e-03, 8.5970e-01,
         6.9320e-01],
        [       nan,        nan,        nan,  ...,        nan,        nan,
                nan]])
CEL: tensor([[2.9504e-01, 5.4787e-01, 1.0266e-02,  ..., 2.6228e-03, 1.6895e+00,
         4.9511e+00],
        [3.8211e-01, 4.5350e-02, 8.7911e-03,  ..., 2.3457e-03, 3.8079e+00,
         4.5770e+00],
        [4.6728e-02, 5.5428e+00, 3.8723e+00,  ..., 9.5806e+00, 1.2351e-02,
         1.0193e-01],
        ...,
        [4.1483e-01, 2.6222e-02, 1.3407e-02,  ..., 1.4764e-03, 1.9640e+00,
         1.1

# BPNN Using Pytorch - November 3, 2022

In [61]:
import torch
from torch.autograd import Variable

In [62]:
w = Variable(torch.Tensor([1.0]), requires_grad=True)

In [16]:
def forward(x):
    return x*w

In [17]:
def loss(x, y):
    y_pred = forward(x)
    return (y_pred - y) * (y_pred - y)

### columns to dataframe

In [20]:
type(x_normalized)
type(y)

pandas.core.frame.DataFrame

In [39]:
# transform dataframe into list
x_tolist = x_normalized.values.tolist()
x_tolist

[[0.0,
  0.0,
  0.02127659574468085,
  1.0,
  0.0,
  0.0,
  0.6666666666666666,
  1.0,
  0.0,
  0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.6666666666666666, 1.0, 0.0, 0.0],
 [0.0,
  0.1717336117740216,
  0.0,
  0.0,
  1.0,
  0.3333333333333333,
  0.0,
  1.0,
  0.16475149233397632,
  1.0],
 [0.0,
  0.7052370337964847,
  0.2765957446808511,
  1.0,
  1.0,
  0.6666666666666666,
  0.6666666666666666,
  0.0,
  0.3980955232362153,
  1.0],
 [0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.6666666666666666,
  0.6666666666666666,
  0.0,
  0.0,
  0.0],
 [0.0,
  0.04704821166057467,
  0.0,
  0.0,
  1.0,
  0.6666666666666666,
  0.0,
  0.0,
  0.04123585630250239,
  0.0],
 [0.0,
  0.04256061118295766,
  0.0,
  1.0,
  1.0,
  0.6666666666666666,
  0.0,
  0.0,
  0.03880492878959787,
  0.0],
 [0.0,
  0.22018069593724565,
  0.0,
  0.0,
  1.0,
  0.6666666666666666,
  0.0,
  0.0,
  0.19297948247824626,
  1.0],
 [0.0,
  1.0,
  0.06382978723404255,
  1.0,
  1.0,
  0.6666666666666666,
  0.0,
  0.0,
  1.0,
  1.0],
 [1.0,
  0

In [48]:
# y_tolist = y.values.tolist()
# y_tolist
# https://stackoverflow.com/questions/34238391/convert-one-dataframe-row-to-flat-list
y_tolist = y.values.flatten().tolist()
y_tolist

[0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


### pytorch

In [68]:
X = torch.tensor(x_normalized.values, dtype=torch.float)
y = torch.tensor(y.values, dtype=torch.float)

In [69]:
# We only take 2 classes to make a binary classification problem
X = X[:y[y < 2].size()[0]]
y = y[:y[y < 2].size()[0]]

print(X.size())
print(y.size())

torch.Size([857, 10])
torch.Size([857, 1])


In [73]:
type(X)

torch.Tensor

### Model Application

In [78]:
print("predict (before training)",4,forward(4).data[0])

for epoch in range(100):
    for x_val, y_val in zip(X, y):
        l = loss(x_val, y_val)
        print("This is l: ", l)
        l.mean().backward()
        # The backward() automatically computes and 
        # Updates the gradient for all variables who has requires_grad=True set True
        # Every time a variable is back propogated through,
        # the gradient will be accumulated instead of being replaced. 
        print("grad: ",x_val ,y_val ,w.grad.data[0])
        w.data = w.data - 0.01*w.grad.data
            
        # set it to zero manually after every batch    
        w.grad.data.zero_()

    print("progress:", epoch, l.data[0])
    print("\n")


print("predict (after training)",  4, forward(4).data[0])

predict (before training) 4 tensor(4.)
This is l:  tensor([0.0000e+00, 0.0000e+00, 4.5269e-04, 1.0000e+00, 0.0000e+00, 0.0000e+00,
        4.4444e-01, 1.0000e+00, 0.0000e+00, 0.0000e+00],
       grad_fn=<MulBackward0>)
